In [ ]:
!pip install torchkge

In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import cosine_similarity
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from torchkge.models import ConvKBModel
import pickle 
from copy import deepcopy
from tqdm import tqdm 
from kge_train import train_kge
from torchkge.utils.datasets import load_fb15k

import transformers
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

torch.backends.cudnn.benchmark = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [101]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [102]:
HOME_DIR = "./"

In [103]:
#df = pd.read_csv("./datasets/20220920_csv/병원_triple.csv", encoding='cp949')
#df[530:550]

In [104]:
#df = pd.read_csv("./datasets/04 경찰서 병원 소방서/소방서_triple.csv", encoding='cp949')
FOLDER_NAME = "datasets/20220920_csv/"

In [105]:
path_to_file_folder = HOME_DIR+FOLDER_NAME
meta_data= ["file_name", "colum_name", "value_list"] #the meta-data should be extented based on EDA on the datasets
df_full = pd.DataFrame(columns=meta_data)
meta_data= ["head", "relation", "tail", "file_name"] #the meta-data should be extented based on EDA on the datasets
kg_full = pd.DataFrame(columns=meta_data)

for file_name in [file for file in os.listdir(path_to_file_folder)]:
    file_name = file_name
    a_file = HOME_DIR+FOLDER_NAME+file_name
    print("Reading the file: ", a_file)
    
    try:
        a_df = pd.DataFrame()
        if ".csv" in file_name:
            a_df = pd.read_csv(a_file, encoding='cp949', names=meta_data, header=None)
            a_df["file_name"] = file_name
        elif ".xlsx" in file_name:
            a_df = pd.read_excel(a_file, names=meta_data, header=None)
            a_df["file_name"] = file_name
        else:
            continue
        kg_full = kg_full.append(a_df, ignore_index=True)
        

        file_name_list = []
        colum_name_list = []
        value_list = []
        for col in a_df.columns:
            col_name = col
            col_values = list(a_df[col_name].values)
            file_name_list.append(file_name)
            colum_name_list.append(col_name)
            value_list.append(col_values)

        tmp_df = pd.DataFrame({"file_name": file_name_list,
                            "colum_name": colum_name_list,
                            "value_list": value_list})

        df_full = df_full.append(tmp_df, ignore_index=True)
        
    except:
        print("읽으려는 파일의 포멧을 확인해야함, 파일명: ", a_file,)


Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)07.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)06.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)02.csv
Reading the file:  ./datasets/20220920_csv/02 해운대_배수펌프(3075794).csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)05.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation).xlsx
읽으려는 파일의 포멧을 확인해야함, 파일명:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation).xlsx
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)04.csv
Reading the file:  ./datasets/20220920_csv/병원_triple.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)03.csv
Reading the file:  ./datasets/20220920_csv/00 general_triple.csv
Reading the file:  ./datasets/20220920_csv/소방서_triple.csv
Reading the file:  ./datasets/20220920_csv/01 부산_유수

/tmp/ipykernel_2947622/4084913395.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kg_full = kg_full.append(a_df, ignore_index=True)
/tmp/ipykernel_2947622/4084913395.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full = df_full.append(tmp_df, ignore_index=True)
/tmp/ipykernel_2947622/4084913395.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kg_full = kg_full.append(a_df, ignore_index=True)
/tmp/ipykernel_2947622/4084913395.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full = df_full.append(tmp_df, ignore_index=True)
/tmp/ipykernel_2947622/4084913395.py:22: FutureWarning: The frame.append method is deprecate

In [106]:
kg_full

,head,relation,tail,file_name
0,7633,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
1,7634,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
2,7635,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
3,7636,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
4,7637,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
...,...,...,...,...
10438,광남지구대,latitude,35.14537016,경찰서_triple.csv
10439,망미2치안센터,latitude,35.1717912,경찰서_triple.csv
10440,부산사상경찰서,latitude,35.17452651,경찰서_triple.csv
10441,부산기장경찰서,latitude,35.32481808,경찰서_triple.csv


In [107]:
class Config:
    kge_n_epoch = 100
    kge_lr = 1e-4
    kge_batch = 64
    kge_margin = 0.5
    kge_conv_size = 3

In [108]:
config = Config()

In [109]:
#kg_full[["head", "relation", "tail"]].to_csv("triple_kb2.csv")
kg_full = kg_full.astype(str)

In [110]:
kb_emb_path = './kge_save'
os.makedirs(kb_emb_path, exist_ok=True)

In [111]:
#from torchkge.utils.datasets import load_fb15k
#kg, kg_val, kg_test = load_fb15k()
#
#model_convKB = ConvKBModel(256,
#                            3,
#                            kg.n_ent,
#                            kg.n_rel,
#)

In [112]:
kg, model_convKB = train_kge(kg_full[["head", "relation", "tail"]],
                             epochs=config.kge_n_epoch, 
                                lr=config.kge_lr, 
                                batch_size=config.kge_batch,
                                margin=config.kge_margin,
                                conv_size=config.kge_conv_size
                                )

originally counts 10443

user nunique 1250, item nunique 2457


Epoch 100 | mean loss: 6.70930: 100%|██████████| 100/100 [00:42<00:00,  2.34epoch/s]


In [113]:
emb_entity_ = model_convKB.get_embeddings()[0].detach().cpu().numpy()
emb_rel_ = model_convKB.get_embeddings()[1].detach().cpu().numpy()

ent_emb_dim = len(emb_entity_[0])
ent_vocab={}
ent_vocab['<unk>'] = 0
ent_vocab['<pad>'] = 1
ent_vocab.update({word: i+2 for word, i in kg.ent2ix.items()})
ent_ids = {i: word for word, i in ent_vocab.items()}

pad_emb_npa = np.zeros((1, ent_emb_dim))   #embedding for '<pad>' token.
unk_emb_npa = np.zeros((1, ent_emb_dim))   #embedding for '<unk>' token.

ent_embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,emb_entity_))
print(ent_embs_npa.shape)
ent_emb = torch.nn.Embedding.from_pretrained(torch.from_numpy(ent_embs_npa).float(), freeze=False)

with open('ent_vocab_npa.npy','wb') as f:
    np.save(f,ent_vocab)

with open('ent_embs_npa.npy','wb') as f:
    np.save(f,ent_emb)


(3705, 256)


In [114]:
print(ent_emb(torch.LongTensor([10, 15]))) 
ids = ent_vocab["부곡지구대"]
print(ent_emb(torch.LongTensor([ids])))

tensor([[ 2.3021e-02,  1.9692e-02, -3.0930e-02,  3.0765e-02, -1.1452e-02,
         -3.4523e-02,  2.2705e-02,  2.3650e-03, -5.7116e-03,  2.0812e-02,
          7.2303e-03, -4.8079e-03, -1.4202e-02,  1.9260e-02, -4.3831e-03,
          2.5393e-02, -1.5948e-03, -2.2197e-02,  4.4254e-04,  3.9051e-03,
          1.4461e-02,  2.3048e-02, -2.5585e-02, -1.2434e-03,  9.9712e-03,
          3.0517e-02,  1.0509e-02,  1.4599e-02,  3.1488e-04,  2.5773e-02,
          3.7756e-02,  1.1176e-02, -3.1508e-03, -3.2699e-02,  3.3879e-02,
          2.2055e-02,  2.9955e-02, -2.5195e-02,  1.1792e-02, -3.4177e-03,
          2.8374e-02,  1.6158e-02, -3.2340e-03, -2.0495e-02,  3.7520e-02,
          1.1076e-03,  5.5587e-03,  1.7482e-02, -2.4807e-03, -3.4690e-02,
         -5.3119e-03,  3.1779e-02, -3.2209e-02, -1.1101e-03,  2.0448e-02,
         -2.0217e-02,  2.2409e-03, -8.0148e-03,  1.1387e-03,  4.2913e-03,
         -1.7926e-02,  9.6873e-03, -1.2314e-02, -3.2993e-02,  1.8669e-02,
         -1.8284e-02, -8.2981e-03,  8.

In [116]:
ent_name="부곡지구대"
e1_ids = ent_vocab[ent_name]
e1_torch = ent_emb(torch.LongTensor([e1_ids]))
e2_ids = np.arange(0, len(ent_vocab))
e2_torch = ent_emb(torch.LongTensor([e2_ids])).squeeze()
similarity = cosine_similarity(e1_torch, e2_torch)
s_value, s_ids = torch.topk(similarity, 100)
s_ids = s_ids.squeeze()
s_ids

tensor([3018, 2095, 2653, 2610, 2030, 2606, 1830, 1829, 1827, 2404, 2084, 2209,
        1828, 2162, 2609, 2205, 2557, 2258, 2608, 2446, 2549, 2680, 2612, 2633,
        2409, 2651, 1970, 1968, 2093, 2161, 2054, 2148, 2164, 2089, 2466, 2727,
        1852, 2163, 2585, 2157, 2765, 1804, 2701, 1811, 1861, 2572, 2450, 2693,
        2671, 1755, 2220, 2451, 1996, 2670, 2017, 2168, 2605, 1851, 2170, 2648,
        2223, 2212, 2582, 1781, 2576, 2340, 2599, 3382, 2624, 2564, 2638, 2137,
        2616, 2471, 2602, 2696, 2678, 2677, 2219, 2719, 1767, 2166, 2232, 2580,
        1786, 2072, 2590, 2376, 1826, 2706, 2573, 2560, 3162, 1799, 2611, 2214,
        2548, 2584, 2712, 2150])

In [117]:
for idx in s_ids:
    print(idx)
    print(ent_ids[int(idx)])

tensor(3018)
부곡지구대
tensor(2095)
7991
tensor(2653)
8511
tensor(2610)
8469
tensor(2030)
7926
tensor(2606)
8465
tensor(1830)
7731
tensor(1829)
7730
tensor(1827)
7728
tensor(2404)
8271
tensor(2084)
7980
tensor(2209)
8083
tensor(1828)
7729
tensor(2162)
8036
tensor(2609)
8468
tensor(2205)
8079
tensor(2557)
8416
tensor(2258)
8128
tensor(2608)
8467
tensor(2446)
8309
tensor(2549)
8408
tensor(2680)
8538
tensor(2612)
8471
tensor(2633)
8492
tensor(2409)
8276
tensor(2651)
8509
tensor(1970)
7868
tensor(1968)
7866
tensor(2093)
7989
tensor(2161)
8035
tensor(2054)
7950
tensor(2148)
8022
tensor(2164)
8038
tensor(2089)
7985
tensor(2466)
8329
tensor(2727)
8585
tensor(1852)
7753
tensor(2163)
8037
tensor(2585)
8444
tensor(2157)
8031
tensor(2765)
8623
tensor(1804)
7705
tensor(2701)
8559
tensor(1811)
7712
tensor(1861)
7762
tensor(2572)
8431
tensor(2450)
8313
tensor(2693)
8551
tensor(2671)
8529
tensor(1755)
7659
tensor(2220)
8094
tensor(2451)
8314
tensor(1996)
7894
tensor(2670)
8528
tensor(2017)
7913
tensor(21

In [118]:
def similarity_example_on_TransE(kg, ent_name="부곡지구대"):
    e1 = emb_entity_[kg.ent2ix[ent_name]]
    r1 = emb_entity_[kg.rel2ix['koad:roadNameCode']]
    e2 = emb_entity_[kg.head_idx]
    similarity = cosine_similarity(torch.Tensor([e1]), torch.Tensor(e2))
    s_value, s_ids = torch.topk(similarity, 10)
    for idx in s_ids:
        print(int(idx))
        print(kg.head_idx[int(idx)])
        print(kg.ix2ent[int(kg.head_idx[int(idx)])])

## text to KB entity module
 - input: text (entity name)
 - output: 

In [119]:
class KLDiveEmb(nn.Module):
    def __init__(self, input_dim, hidden_dim, out_dim):
        super(KLDiveEmb, self).__init__()

        self.bert = transformers.XLMRobertaModel.from_pretrained('xlm-roberta-base')
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.drop_out = nn.Dropout(0.33)
        self.fc1 = nn.Linear(hidden_dim, out_dim)

    def forward(self, idx, mask):
    
        x = self.bert(idx, mask) 
        x = x.pooler_output
        x = F.relu(self.layer1(x))
        x = self.drop_out(x)
        x = F.relu(self.fc1(x))
        
        return F.softmax(x, dim=1), x

In [120]:
class KLEDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, data, max_token):
        
        self.tokenizer = tokenizer
        self.data = data
        self.max_token = max_token 
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        e_name = self.data['e_name'][index] #Entity Name
        e_emb = self.data['e_emb'][index]  #KB embedding
            
        tokenized = self.tokenizer.encode_plus("".join(e_name),
                                                None,
                                                add_special_tokens=True,
                                                max_length = self.max_token,
                                                padding='max_length',
                                                truncation=True,
                                              )
        
        ids = tokenized['input_ids']
        mask = tokenized['attention_mask']

        return {'e_ids': torch.tensor(ids, dtype=torch.long), 
                'e_mask': torch.tensor(mask, dtype=torch.long),
                'target': torch.tensor(e_emb)}

In [121]:
model = KLDiveEmb(768, 512, 256).to(DEVICE)
criterion = torch.nn.KLDivLoss(reduction='batchmean')
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005)
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [122]:
k,i,v = [],[],[]
for ent, idx in ent_vocab.items():
    ent_name=ent
    e1_ids = ent_vocab[ent_name]
    e1_torch = ent_emb(torch.LongTensor([e1_ids]))
    k.append(ent)
    i.append(idx)
    v.append(e1_torch.squeeze().detach().numpy())

In [123]:
df = pd.DataFrame(columns=['e_name','e_idx', 'e_emb'])
df["e_name"] = k
df["e_idx"] = i
df["e_emb"] = v

In [124]:
train_df, valid_df = train_test_split(df, test_size=0.2)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
batch_size = 64

In [125]:
train_dataset = KLEDataset(tokenizer, train_df, max_token=20)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=4, shuffle=True, pin_memory=True)
valid_dataset = KLEDataset(tokenizer, valid_df, max_token=20) 
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size , num_workers=4, shuffle=False, pin_memory=True)

In [126]:
def train_fn(model, train_loader, optimizer, criterion):
    model.train()
    batch_loss = []
    for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader), leave=False): #학습 데이터를 batch size만큼씩 읽어옴
        optimizer.zero_grad()
        e_ids = batch['e_ids'].to(DEVICE) #entity_name
        e_bert_mask = batch['e_mask'].to(DEVICE) 
        
        logits, _ = model(e_ids, e_bert_mask) 
                
        target = batch['target'].to(DEVICE) #KB embedding
        target = F.softmax(target, dim=1)
        target.requires_grad = False
        loss = criterion(logits.log(), target)

        batch_loss.append(float(loss.item()))
        loss.backward(loss)
        optimizer.step()
        
    return sum(batch_loss) / len(train_loader)

In [127]:
def valid_fn(model, valid_loader, criterion):
    model.eval()
    batch_loss = []
    with torch.no_grad():
        for idx, batch in tqdm(enumerate(valid_loader), total=len(valid_loader), leave=False): #학습 데이터를 batch size만큼씩 읽어옴
            optimizer.zero_grad()
            e_ids = batch['e_ids'].to(DEVICE) #entity_name
            e_bert_mask = batch['e_mask'].to(DEVICE) 
            
            logits, _ = model(e_ids, e_bert_mask) 
                    
            target = batch['target'].to(DEVICE) #KB embedding
            target = F.softmax(target, dim=1)
            loss = criterion(logits.log(), target)

            batch_loss.append(float(loss.item()))
        
    return sum(batch_loss) / len(valid_loader)

In [128]:
train_epoch_loss = []
test_epoch_loss = []
for epoch in range(5):
    train_avg_loss = train_fn(model, train_loader, optimizer, criterion)
    train_epoch_loss.append(train_avg_loss)
    print(train_avg_loss)

0.0010539988152425182


0.0006732494827914746


0.0006405220111891469


0.0006218416556874488


0.0006122241017131254


In [129]:
valid_avg_loss = valid_fn(model, valid_loader, criterion)
print(valid_avg_loss)

0.0005474132534194117


In [130]:
def prediction(model, keyword="춘천"):

    e_name = keyword
    tokenized = tokenizer.encode_plus("".join(e_name),
                                        None,
                                        add_special_tokens=True,
                                        max_length = 20,
                                        padding='max_length',
                                        truncation=True,
                                        )

    ids = tokenized['input_ids']
    mask = tokenized['attention_mask']

    batch = {'e_ids': torch.tensor(ids, dtype=torch.long), 
             'e_mask': torch.tensor(mask, dtype=torch.long)}

    e_ids = batch['e_ids'].to(DEVICE).unsqueeze(0) #entity_name
    e_bert_mask = batch['e_mask'].to(DEVICE).unsqueeze(0) 
    logits, word_emb = model(e_ids, e_bert_mask)
    e1_torch = word_emb.cpu()
    e2_ids = np.arange(0, len(ent_vocab))
    e2_torch = ent_emb(torch.LongTensor([e2_ids])).squeeze()
    similarity = cosine_similarity(e1_torch, e2_torch)
    s_value, s_ids = torch.topk(similarity, 10)
    s_ids = s_ids.squeeze()
    s_ids

    return s_ids.detach().numpy(), s_value.detach().numpy()

In [131]:
def search_ex_matching(df, query = "<unk>"):
    keyword = query
    exist_boolean = (df["e_name"] == keyword).any()
    ex_matching = 0 #기본적으로 "<unk>"로 해둠
    if exist_boolean: 
        ex_matching = int(df[df["e_name"] == keyword]['e_idx']) 
    else: 
        ex_matching = 0

    return ex_matching


In [132]:
query = "부곡지구대"
ex_ids = search_ex_matching(df, query)
s_ids, s_score = prediction(model, query)
total_ids = np.concatenate([np.array([ex_ids]), s_ids]) 

for idx in total_ids:
    print(df[df['e_idx'] == int(idx)][["e_idx","e_name"]].values.tolist())

[[3018, '부곡지구대']]
[[1836, '7737']]
[[2133, '8007']]
[[1788, '7692']]
[[2629, '8488']]
[[1871, '7772']]
[[1828, '7729']]
[[2163, '8037']]
[[2094, '7990']]
[[2671, '8529']]
[[2752, '8610']]
